In [39]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [40]:
import torch
import tensorflow as tf
import numpy as np

from time import time

pi = tf.constant(np.pi, dtype=tf.float32)

In [41]:
print("Versión de TensorFlow:", tf.__version__)

# Comprueba si TensorFlow fue compilado con soporte para CUDA (GPU)
print("Construido con CUDA:", tf.test.is_built_with_cuda())

# Lista los dispositivos físicos de tipo GPU que TensorFlow puede ver
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"GPUs disponibles: {len(gpus)}")
    for gpu in gpus:
        print(f"  - {gpu}")
    # Opcional: Habilitar el crecimiento de memoria para evitar que TF acapare toda la VRAM
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
else:
    print("¡No se encontró ninguna GPU!")

Versión de TensorFlow: 2.19.0
Construido con CUDA: True
GPUs disponibles: 1
  - PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
Physical devices cannot be modified after being initialized


In [42]:
def torch_fun_1(x):
    if not isinstance(x, torch.Tensor):
        raise ValueError(f"Input must be a pytorch tensor but is {type(x)}")
    if (2,) != x.shape:
        raise ValueError(f"Input must be of shape (2, ) but has shape {x.shape}")
    
    g = torch.Generator()
    g.manual_seed(42)
    y = x + torch.randint(4, 20, size=(2, ), generator=g)
    return sum(y**2)

def torch_fun_2(x):
    if not isinstance(x, torch.Tensor):
        raise ValueError(f"Input must be a pytorch tensor but is {type(x)}")
    if (10,) != x.shape:
        raise ValueError(f"Input must be of shape (10, ) but has shape {x.shape}")
    
    g = torch.Generator()
    g.manual_seed(42)
    y = x + torch.randint(4, 20, size=(10, ), generator=g)
    return sum(y**2)

def tf_fun_1(x):
    if not isinstance(x, tf.Variable):
        raise ValueError(f"Input must be a tensorflow Variable but is {type(x)}")
    if (2,) != x.shape:
        raise ValueError(f"Input must be of shape (2, ) but has shape {x.shape}")
    g = tf.random.Generator.from_seed(42)
    y = x + g.uniform(shape=(2,), minval=4, maxval=20, dtype=tf.float32)
    return sum(y**2)

def tf_fun_2(x):
    if not isinstance(x, tf.Variable):
        raise ValueError(f"Input must be a tensorflow Variable but is {type(x)}")
    if (10,) != x.shape:
        raise ValueError(f"Input must be of shape (10, ) but has shape {x.shape}")
    
    g = tf.random.Generator.from_seed(42)
    y = x + g.uniform(shape=(10,), minval=4, maxval=20, dtype=tf.float32)
    return sum(y**2)


In [43]:
def optimize_torch_fun1(f):
    print("Testing Torch 1")
    """
    Compite con varios optimizadores para encontrar el arg min f y devuelve
    el tensor del optimizador más rápido.
    """
    # Parámetros de la competencia interna
    tolerance = 1e-4
    max_steps = 1000 # Aumentamos los pasos para dar oportunidad a los más lentos
    
    # Estructura para guardar el resultado de cada optimizador
    results = {}
    
    learning_rates = {"SGD": 0.01, "Momentum": 0.01, "Adagrad": 0.1, "RMSprop": 0.01, "Adam": 0.1}
    optimizers_to_test = ["SGD", "Momentum", "Adagrad", "RMSprop", "Adam"]

    print("--- Iniciando competencia en optimize_torch_fun1 ---")
    for opt_name in optimizers_to_test:
        # Reiniciamos la variable 'x' para cada optimizador, para una competencia justa
        x = torch.zeros(2, requires_grad=True)
        lr = learning_rates.get(opt_name, 0.1)

        # Seleccionar el optimizador
        if opt_name == "SGD": optimizer = torch.optim.SGD([x], lr=lr)
        elif opt_name == "Momentum": optimizer = torch.optim.SGD([x], lr=lr, momentum=0.9)
        elif opt_name == "Adagrad": optimizer = torch.optim.Adagrad([x], lr=lr)
        elif opt_name == "RMSprop": optimizer = torch.optim.RMSprop([x], lr=lr)
        elif opt_name == "Adam": optimizer = torch.optim.Adam([x], lr=lr)
        else: continue

        # Bucle de optimización hasta la convergencia
        for step in range(1, max_steps + 1):
            optimizer.zero_grad()
            loss = f(x)
            if loss.item() < tolerance:
                # ¡Importante! Guardamos el número de pasos y una copia del tensor final
                results[opt_name] = {"steps": step, "final_x": x.detach().clone()}
                break
            loss.backward()
            optimizer.step()
        else: # Si el bucle termina sin converger
            results[opt_name] = {"steps": max_steps, "final_x": x.detach().clone()}

    # --- Lógica para decidir el ganador ---
    
    # Filtramos los que sí convergieron
    converged_optimizers = {name: data for name, data in results.items() if data['steps'] < max_steps}
    
    if converged_optimizers:
        # Encontramos el nombre del optimizador más rápido (menos pasos)
        best_optimizer_name = min(converged_optimizers, key=lambda name: converged_optimizers[name]['steps'])
        print(f"-> Ganador de la competencia: {best_optimizer_name} con {results[best_optimizer_name]['steps']} pasos.")
        # Recuperamos el tensor del optimizador más rápido y lo devolvemos
        return results[best_optimizer_name]['final_x']
    else:
        # Fallback: si ninguno convergió, devolvemos el resultado de Adam (suele ser el más robusto)
        print("-> Ningún optimizador convergió, devolviendo el resultado de Adam.")
        return results['Adam']['final_x']

def optimize_torch_fun2(f):
    print("Testing Torch 2")
    """
    Encuentra arg min f utilizando PyTorch.

    Args:
        f: una función que recibe un tensor de PyTorch de forma (10,) y devuelve un float.
    
    Return: un tensor de PyTorch de forma (10,)
    """
        # Parámetros de la competencia interna
    tolerance = 1e-4
    max_steps = 1000 # Aumentamos los pasos para dar oportunidad a los más lentos
    
    # Estructura para guardar el resultado de cada optimizador
    results = {}
    
    learning_rates = {"SGD": 0.01, "Momentum": 0.01, "Adagrad": 0.1, "RMSprop": 0.01, "Adam": 0.1}
    optimizers_to_test = ["SGD", "Momentum", "Adagrad", "RMSprop", "Adam"]

    print("--- Iniciando competencia en optimize_torch_fun2 ---")
    for opt_name in optimizers_to_test:
        # Reiniciamos la variable 'x' para cada optimizador, para una competencia justa
        x = torch.zeros(10, requires_grad=True)
        lr = learning_rates.get(opt_name, 0.1)

        # Seleccionar el optimizador
        if opt_name == "SGD": optimizer = torch.optim.SGD([x], lr=lr)
        elif opt_name == "Momentum": optimizer = torch.optim.SGD([x], lr=lr, momentum=0.9)
        elif opt_name == "Adagrad": optimizer = torch.optim.Adagrad([x], lr=lr)
        elif opt_name == "RMSprop": optimizer = torch.optim.RMSprop([x], lr=lr)
        elif opt_name == "Adam": optimizer = torch.optim.Adam([x], lr=lr)
        else: continue

        # Bucle de optimización hasta la convergencia
        for step in range(1, max_steps + 1):
            optimizer.zero_grad()
            loss = f(x)
            if loss.item() < tolerance:
                # ¡Importante! Guardamos el número de pasos y una copia del tensor final
                results[opt_name] = {"steps": step, "final_x": x.detach().clone()}
                break
            loss.backward()
            optimizer.step()
        else: # Si el bucle termina sin converger
            results[opt_name] = {"steps": max_steps, "final_x": x.detach().clone()}

    # --- Lógica para decidir el ganador ---
    
    # Filtramos los que sí convergieron
    converged_optimizers = {name: data for name, data in results.items() if data['steps'] < max_steps}
    
    if converged_optimizers:
        # Encontramos el nombre del optimizador más rápido (menos pasos)
        best_optimizer_name = min(converged_optimizers, key=lambda name: converged_optimizers[name]['steps'])
        print(f"-> Ganador de la competencia: {best_optimizer_name} con {results[best_optimizer_name]['steps']} pasos.")
        # Recuperamos el tensor del optimizador más rápido y lo devolvemos
        return results[best_optimizer_name]['final_x']
    else:
        # Fallback: si ninguno convergió, devolvemos el resultado de Adam (suele ser el más robusto)
        print("-> Ningún optimizador convergió, devolviendo el resultado de Adam.")
        return results['Adam']['final_x']


def _optimize_tf(f, shape, tolerance, max_steps):
    """Función auxiliar que ejecuta la competencia en modo Eager."""
    results = {}
    learning_rates = {"SGD": 0.01, "Momentum": 0.01, "Adagrad": 0.1, "RMSprop": 0.01, "Adam": 0.1}
    optimizers_to_test = ["SGD", "Momentum", "Adagrad", "RMSprop", "Adam"]

    print("--- Iniciando competencia en _optimize_tf ---")
    for opt_name in optimizers_to_test:
        x = tf.Variable(tf.zeros(shape), dtype=tf.float32)
        lr = learning_rates.get(opt_name, 0.1)

        if opt_name == "SGD": optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
        elif opt_name == "Momentum": optimizer = tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9)
        elif opt_name == "Adagrad": optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr)
        elif opt_name == "RMSprop": optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
        elif opt_name == "Adam": optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
        else: continue

        for step in range(1, max_steps + 1):
            with tf.GradientTape() as tape:
                loss = f(x)
            
            if loss < tolerance:
                results[opt_name] = {"steps": step, "final_x": x}
                break
            
            gradients = tape.gradient(loss, [x])
            
            if gradients[0] is not None:
                optimizer.apply_gradients(zip(gradients, [x]))
            else:
                break
        else:
            results[opt_name] = {"steps": max_steps, "final_x": x}

    # ----- ¡CAMBIOS AQUÍ! Se añaden los prints solicitados -----
    converged_optimizers = {name: data for name, data in results.items() if data['steps'] < max_steps}
    if converged_optimizers:
        best_optimizer_name = min(converged_optimizers, key=lambda name: converged_optimizers[name]['steps'])
        print(f"-> Ganador de la competencia: {best_optimizer_name} con {results[best_optimizer_name]['steps']} pasos.")
        return results[best_optimizer_name]['final_x']
    else:
        print("-> Ningún optimizador convergió, devolviendo el resultado de Adam.")
        return results['Adam']['final_x']

def optimize_tf_fun1(f):
    return _optimize_tf(f, shape=(2,), tolerance=1e-4, max_steps=1000)

def optimize_tf_fun2(f):
    return _optimize_tf(f, shape=(10,), tolerance=1e-4, max_steps=1000)

In [44]:
def score_fun(fun, optimization_fun, optimal_score, minimal_score_for_points, timeout):

    try:

        # run optimizer
        t_start = time()
        params = optimization_fun(fun)
        runtime = time() - t_start
        if runtime > timeout:
            print(f"Optimizer used {runtime}s, but only {timeout} are allowed")

        # determine performance
        funval = fun(params)
        if funval > minimal_score_for_points:
            return 0.0
        else:
            return np.round(float((100 * (minimal_score_for_points - funval) / (minimal_score_for_points - optimal_score))), 1)
        
    except Exception:
        raise

# run optimizer on first task
leaderboard_scores = []

scores = []
for name, obj_fun, optimizer_fun, least_possible_fun_val, fun_val_to_receive_points in [
    ("PyTorch Function 1", torch_fun_1, optimize_torch_fun1, 0, 100),
    ("PyTorch Function 2", torch_fun_2, optimize_torch_fun2, 0, 100),
    ("TensorFlow Function 1", tf_fun_1, optimize_tf_fun1, 0, 100),
    ("TensorFlow Function 2", tf_fun_2, optimize_tf_fun2, 0, 100)
]:
    score = score_fun(obj_fun, optimizer_fun, least_possible_fun_val, fun_val_to_receive_points, timeout=30)
    leaderboard_scores.append({
        "name": name,
        "value": score
    })
    scores.append(score)

Testing Torch 1
--- Iniciando competencia en optimize_torch_fun1 ---
-> Ganador de la competencia: Momentum con 38 pasos.
Testing Torch 2
--- Iniciando competencia en optimize_torch_fun2 ---
-> Ganador de la competencia: Momentum con 108 pasos.
--- Iniciando competencia en _optimize_tf ---
-> Ganador de la competencia: Momentum con 107 pasos.
Optimizer used 37.966572761535645s, but only 30 are allowed
--- Iniciando competencia en _optimize_tf ---
-> Ganador de la competencia: Momentum con 108 pasos.
Optimizer used 58.91860055923462s, but only 30 are allowed


In [45]:
print(scores)

[np.float64(100.0), np.float64(100.0), np.float64(100.0), np.float64(100.0)]
